In [ ]:
import torch
import torchvision
from torchvision.datasets import FakeData
from torchvision import transforms
import matplotlib.pyplot as plt
import torch.nn.functional as F
from torch import nn
from torch import optim
import numpy as np
from sklearn import metrics

In [ ]:

transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

batch_size = 256

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                         shuffle=False)

fakeset = FakeData(size=1000, image_size=(3, 32, 32), transform=transform)
fakeloader = torch.utils.data.DataLoader(fakeset, batch_size=batch_size,
                                         shuffle=False)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

In [ ]:
# A very simple CNN model.
class CNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [ ]:
# USE THIS CELL TO LOAD THE PRETRAINED MODEL.
device = 'cpu'
loss = nn.CrossEntropyLoss()
model = CNN().to(device)
model.load_state_dict(torch.load('./OOD_CNN.pth',map_location=torch.device('cpu'))) #50 epochs

In [ ]:
def max_softmax(logit, T=1.0, delta= 0.5):
    s = F.softmax(logit/T, 1)
    s = s.max(dim=1)[0] #get the max for each element of the batch
    return s
    


def compute_scores(data_loader, score_fun):
    scores = []
    with torch.no_grad():
        for data in data_loader:
            x, y = data
            output = model(x.to(device))
            s = score_fun(output)
            scores.append(s)
        scores_t = torch.cat(scores)
        return scores_t


def OOD_det(model, data_loader, T, eps, delta):
    score = []
    for data in data_loader:
        model.train()
        x,y = data
        
        x.requires_grad=True 

        x.retain_grad()

        output=model(x)
        model.zero_grad()
        l = loss(output,y)
        l.backward()
        v = x + eps*torch.sign(x.grad)#fgsm
        with torch.no_grad():
            logit = model(v)
            s = max_softmax(logit, T, delta)
            for i in range(np.shape(s)[0]): #calcolo della funzione g
                if s[i]>delta:
                    score.append(1)
                else:
                    score.append(0)
    score_t = torch.tensor(score)
    return score_t






In [ ]:
##GRID SEARCH ON EPS, T, DELTA(?). Uso test set come validation set.


e = np.linspace(0, 0.004, 10)#10 valori da 0 a 0.004 come nel paper su ODIN
t = np.array([1, 5, 10, 50, 100, 500, 1000])

for eps in e:
    for temp in t:
        scores_fake= OOD_det(model, fakeloader, temp, eps, delta=0.5)
        scores_test= OOD_det(model, testloader, temp, eps, delta=0.5)
        y_test = torch.ones_like(scores_test)
        y_fake = torch.zeros_like(scores_fake)
        y = torch.cat((y_test, y_fake))
        y_pred = torch.cat((scores_test, scores_fake))
        s = metrics.roc_auc_score(y, y_pred)
        print("AUC for eps = %s, temp = %s is %s" % (eps, temp, s))

